In [2]:
pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install SQLAlchemy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 22.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install scikit-learn

  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 25.4 MB/s eta 0:00:00a 0:00:01
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.1/30.1 MB 26.6 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine
from sklearn.metrics.pairwise import cosine_similarity

In [71]:
# データベースの接続情報
DB_USER = "ryoueno"
DB_PASSWORD = "tech0-himdb"
DB_HOST = "tech0-gen-8-step4-him-database.mysql.database.azure.com"
DB_PORT = 3306
DB_NAME = "hsp-db"

In [149]:
# MySQLに接続
engine = create_engine(f"mysql+mysqlconnector://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

#### Usersテーブルの情報取得

In [162]:
# SQLクエリ
query_users = "SELECT user_id, gender, relationship_id, area_id, birth_date FROM Users;"

In [163]:
df_users = pd.read_sql(query_users, engine)
df_users["gender"] = df_users["gender"].map({"M": 0, "F": 1, "U": 2})

In [164]:
print(df_users)

    user_id  gender  relationship_id  area_id  birth_date
0         1       0                1        1  1955-06-10
1         2       0                1        3  1951-03-22
2         3       0                1        2  1948-09-15
3         4       1                2        5  1957-12-01
4         5       1                2        4  1949-04-25
5         6       0                1        6  1953-10-30
6         7       1                2        7  1958-08-12
7         8       0                1        8  1952-01-05
8         9       1                2        9  1955-11-18
9        10       0                1       10  1975-07-20
10       11       1                2        1  1980-03-14
11       12       0                1        3  1968-12-22
12       13       1                2        2  1972-06-05
13       14       0                1        5  1979-09-30
14       15       1                2        4  1965-04-18
15       16       0                3        6  1982-08-21
16       17   

In [165]:
# 年齢の計算
df_users["age"] = pd.to_datetime("today").year - pd.to_datetime(df_users["birth_date"]).dt.year
df_users.drop(columns=["birth_date"], inplace=True)

In [166]:
# 年齢の画像表示 Min-Maxスケーリング
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df_users["age"] = scaler.fit_transform(df_users[["age"]])

In [167]:
print(df_users.dtypes)

user_id              int64
gender               int64
relationship_id      int64
area_id              int64
age                float64
dtype: object


In [168]:
print(df_users["age"].describe())
print(df_users["age"].unique()) 

count    30.000000
mean      0.542188
std       0.313929
min       0.000000
25%       0.273438
50%       0.531250
75%       0.855469
max       1.000000
Name: age, dtype: float64
[0.890625 0.953125 1.       0.859375 0.984375 0.921875 0.84375  0.9375
 0.578125 0.5      0.6875   0.625    0.515625 0.734375 0.46875  0.546875
 0.484375 0.65625  0.296875 0.25     0.328125 0.21875  0.234375 0.1875
 0.359375 0.265625 0.03125  0.       0.015625]


In [169]:
df_users

,user_id,gender,relationship_id,area_id,age
0,1,0,1,1,0.890625
1,2,0,1,3,0.953125
2,3,0,1,2,1.000000
3,4,1,2,5,0.859375
4,5,1,2,4,0.984375
5,6,0,1,6,0.921875
6,7,1,2,7,0.843750
7,8,0,1,8,0.937500
8,9,1,2,9,0.890625
9,10,0,1,10,0.578125


In [170]:
df_users = pd.get_dummies(df_users, columns=["area_id"])

In [172]:
area_columns = [col for col in df_users.columns if col.startswith("area_id_")]
df_users[area_columns] = df_users[area_columns].astype(int)

In [173]:
df_users

,user_id,gender,relationship_id,age,area_id_1,area_id_2,area_id_3,area_id_4,area_id_5,area_id_6,area_id_7,area_id_8,area_id_9,area_id_10
0,1,0,1,0.890625,1,0,0,0,0,0,0,0,0,0
1,2,0,1,0.953125,0,0,1,0,0,0,0,0,0,0
2,3,0,1,1.000000,0,1,0,0,0,0,0,0,0,0
3,4,1,2,0.859375,0,0,0,0,1,0,0,0,0,0
4,5,1,2,0.984375,0,0,0,1,0,0,0,0,0,0
5,6,0,1,0.921875,0,0,0,0,0,1,0,0,0,0
6,7,1,2,0.843750,0,0,0,0,0,0,1,0,0,0
7,8,0,1,0.937500,0,0,0,0,0,0,0,1,0,0
8,9,1,2,0.890625,0,0,0,0,0,0,0,0,1,0
9,10,0,1,0.578125,0,0,0,0,0,0,0,0,0,1


In [174]:
# 変換後のデータ確認
print(df_users["age"].describe())
print(df_users["age"].unique())

count    30.000000
mean      0.542188
std       0.313929
min       0.000000
25%       0.273438
50%       0.531250
75%       0.855469
max       1.000000
Name: age, dtype: float64
[0.890625 0.953125 1.       0.859375 0.984375 0.921875 0.84375  0.9375
 0.578125 0.5      0.6875   0.625    0.515625 0.734375 0.46875  0.546875
 0.484375 0.65625  0.296875 0.25     0.328125 0.21875  0.234375 0.1875
 0.359375 0.265625 0.03125  0.       0.015625]


#### UserTags情報の取得

In [175]:
query_tags = """
SELECT u.user_id, t.tag_id
FROM UserTags u
JOIN Tags t ON u.tag_id = t.tag_id;
"""

SQL構文	説明
SELECT u.user_id, t.tag_id	user_id（ユーザー）と tag_id（タグ）を取得
FROM UserTags u	UserTags テーブルを u という別名で参照
JOIN Tags t	Tags テーブルを t という別名で参照し、UserTags と結合
ON u.tag_id = t.tag_id	UserTags の tag_id と Tags の tag_id が一致する行を取得

In [176]:
df_tags = pd.read_sql(query_tags, engine)
df_tags

,user_id,tag_id
0,7,8
1,13,8
2,19,8
3,27,8
4,2,3
...,...,...
81,22,7
82,5,10
83,11,10
84,19,10


In [178]:
# ワンホットエンコーディング
df_tags_onehot = df_tags.pivot_table(index="user_id", columns="tag_id", aggfunc="size", fill_value=0)
df_tags_onehot = df_tags_onehot.add_prefix("tag_")
df_tags_onehot

tag_id,tag_1,tag_2,tag_3,tag_4,tag_5,tag_6,tag_7,tag_8,tag_9,tag_10,...,tag_17,tag_18,tag_19,tag_20,tag_21,tag_22,tag_23,tag_24,tag_25,tag_26
user_id,,,,,,,,,,,,,,,,,,,,,
1,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
6,0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,1,1,1,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,0


#### PointTransaction情報の取得

In [179]:
query_transactions = "SELECT user_id, store_id FROM pointtransaction WHERE user_id IS NOT NULL;"
df_pointtransaction = pd.read_sql(query_transactions, engine)
df_pointtransaction

,user_id,store_id
0,1,1
1,2,2
2,3,3
3,4,4
4,5,5
5,6,6
6,1,1
7,2,2
8,3,3
9,4,4


In [180]:
# ワンホットエンコーディング
df_transactions_onehot = df_pointtransaction.pivot_table(index="user_id", columns="store_id", aggfunc="size", fill_value=0)
df_transactions_onehot = df_transactions_onehot.add_prefix("store_")
df_transactions_onehot

store_id,store_1,store_2,store_3,store_4,store_5,store_6
user_id,,,,,,
1,3,0,0,0,0,0
2,0,3,0,0,0,0
3,1,0,3,0,1,0
4,1,0,0,3,1,0
5,0,0,0,1,3,1
6,0,0,0,1,0,4
7,0,1,0,0,1,0
8,0,0,1,1,0,0
9,0,0,2,0,0,0


#### データの統合

In [181]:
df_final = df_users.set_index("user_id")\
    .join(df_tags_onehot, how="left")\
    .join(df_transactions_onehot, how="left")\
    .fillna(0)

In [182]:
print(df_final.head())

         gender  relationship_id       age  area_id_1  area_id_2  area_id_3  \
user_id                                                                       
1             0                1  0.890625          1          0          0   
2             0                1  0.953125          0          0          1   
3             0                1  1.000000          0          1          0   
4             1                2  0.859375          0          0          0   
5             1                2  0.984375          0          0          0   

         area_id_4  area_id_5  area_id_6  area_id_7  ...  tag_23  tag_24  \
user_id                                              ...                   
1                0          0          0          0  ...       0       1   
2                0          0          0          0  ...       0       0   
3                0          0          0          0  ...       0       0   
4                0          1          0          0  ...       0  

In [188]:
df_final.to_excel("~/Desktop/users_data.xlsx", index=True, engine="openpyxl")


In [3]:
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine
from sklearn.metrics.pairwise import cosine_similarity

# データベースの接続情報
DB_USER = "ryoueno"
DB_PASSWORD = "tech0-himdb"
DB_HOST = "tech0-gen-8-step4-him-database.mysql.database.azure.com"
DB_PORT = 3306
DB_NAME = "hsp-db"

# MySQLに接続
engine = create_engine(f"mysql+mysqlconnector://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

# Usersテーブルの情報取得
query_users = "SELECT user_id, gender, relationship_id, area_id, birth_date FROM Users;"
df_users = pd.read_sql(query_users, engine)
df_users["gender"] = df_users["gender"].map({"M": 0, "F": 1, "U": 2})

# 年齢の計算
df_users["age"] = pd.to_datetime("today").year - pd.to_datetime(df_users["birth_date"]).dt.year
df_users.drop(columns=["birth_date"], inplace=True)

# 年齢の画像表示 Min-Maxスケーリング
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df_users["age"] = scaler.fit_transform(df_users[["age"]])

df_users = pd.get_dummies(df_users, columns=["area_id"])
area_columns = [col for col in df_users.columns if col.startswith("area_id_")]
df_users[area_columns] = df_users[area_columns].astype(int)


#UserTags情報の取得
query_tags = """SELECT u.user_id, t.tag_id FROM UserTags u JOIN Tags t ON u.tag_id = t.tag_id;"""
df_tags = pd.read_sql(query_tags, engine)
# ワンホットエンコーディング
df_tags_onehot = df_tags.pivot_table(index="user_id", columns="tag_id", aggfunc="size", fill_value=0)
df_tags_onehot = df_tags_onehot.add_prefix("tag_")

#PointTransaction情報の取得
query_transactions = "SELECT user_id, store_id FROM pointtransaction WHERE user_id IS NOT NULL;"
df_pointtransaction = pd.read_sql(query_transactions, engine)
# ワンホットエンコーディング
df_transactions_onehot = df_pointtransaction.pivot_table(index="user_id", columns="store_id", aggfunc="size", fill_value=0)
df_transactions_onehot = df_transactions_onehot.add_prefix("store_")


#データの統合
df_final = df_users.set_index("user_id")\
    .join(df_tags_onehot, how="left")\
    .join(df_transactions_onehot, how="left")\
    .fillna(0)

print(df_final.head())


         gender  relationship_id       age  area_id_1  area_id_2  area_id_3  \
user_id                                                                       
1             0                1  0.890625          1          0          0   
2             0                1  0.953125          0          0          1   
3             0                1  1.000000          0          1          0   
4             1                2  0.859375          0          0          0   
5             1                2  0.984375          0          0          0   

         area_id_4  area_id_5  area_id_6  area_id_7  ...  tag_23  tag_24  \
user_id                                              ...                   
1                0          0          0          0  ...       0       1   
2                0          0          0          0  ...       0       0   
3                0          0          0          0  ...       0       0   
4                0          1          0          0  ...       0  

In [4]:
# コサイン類似度の計算
cosine_sim = cosine_similarity(df_final)
# 結果をデータフレームに変換
cosine_sim_df = pd.DataFrame(cosine_sim, index=df_final.index, columns=df_final.index)
# コサイン類似度行列の表示
print(cosine_sim_df)


user_id        1         2         3         4         5         6         7   \
user_id                                                                         
1        1.000000  0.128895  0.308396  0.337396  0.163334  0.099048  0.218574   
2        0.128895  1.000000  0.127018  0.170142  0.172051  0.105382  0.475521   
3        0.308396  0.127018  1.000000  0.265276  0.316961  0.097512  0.284837   
4        0.337396  0.170142  0.265276  1.000000  0.582269  0.272739  0.462496   
5        0.163334  0.172051  0.316961  0.582269  1.000000  0.361249  0.589205   
6        0.099048  0.105382  0.097512  0.272739  0.361249  1.000000  0.177554   
7        0.218574  0.475521  0.284837  0.462496  0.589205  0.177554  1.000000   
8        0.169966  0.180886  0.426628  0.465545  0.305197  0.213470  0.303806   
9        0.203041  0.213572  0.602862  0.362811  0.358803  0.164998  0.491309   
10       0.128917  0.136126  0.125279  0.183947  0.326589  0.652750  0.248795   
11       0.344584  0.197986 

In [191]:
cosine_sim_df.to_excel("~/Desktop/cosine_sim_df.xlsx", index=True, engine="openpyxl")

#### イベント情報を届けるロジック

In [5]:
# 類似ユーザーの抽出
def get_similar_users(user_id, top_n=5):
    return cosine_sim_df[user_id].sort_values(ascending=False).iloc[1:top_n+1]

In [21]:
#類似ユーザーの抽出test
target_user_id = 1 #ログインしているユーザー
similar_users = get_similar_users(target_user_id).index
similar_users


Index([17, 19, 22, 16, 21], dtype='int64', name='user_id')

In [22]:
# 類似ユーザーが訪問した店舗・イベントを集計
similar_users = get_similar_users(target_user_id).index
existing_users = [user for user in similar_users if user in df_transactions_onehot.index] # df_transactions_onehot に存在するユーザーだけフィルタ
if existing_users:
    recommended_stores = df_transactions_onehot.loc[existing_users].sum().sort_values(ascending=False)
    print(f"ユーザー {target_user_id} におすすめの店舗:")
    print(recommended_stores.head())
else:
    print(f"ユーザー {target_user_id} に類似ユーザーの来店履歴がありません。")

ユーザー 1 におすすめの店舗:
store_id
store_1    4
store_2    1
store_3    1
store_4    1
store_5    1
dtype: int64


In [17]:
similar_users

Index([19, 20, 29, 3, 30], dtype='int64', name='user_id')

In [16]:
existing_users

[19, 20, 3]

In [ ]:
# 類似ユーザーのイベント履歴の取得:
def get_similar_users_events(user_id, cosine_sim_df, event_history_df, top_n=5):
    similar_users = cosine_sim_df[user_id].sort_values(ascending=False).index[1:top_n+1]
    similar_users_events = event_history_df[event_history_df['user_id'].isin(similar_users)]
    return similar_users_events


In [1]:
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine
from sklearn.metrics.pairwise import cosine_similarity

# データベースの接続情報
DB_USER = "ryoueno"
DB_PASSWORD = "tech0-himdb"
DB_HOST = "tech0-gen-8-step4-him-database.mysql.database.azure.com"
DB_PORT = 3306
DB_NAME = "hsp-db"

# MySQLに接続
engine = create_engine(f"mysql+mysqlconnector://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

# Usersテーブルの情報取得
query_users = "SELECT user_id, gender, relationship_id, postal_code, birth_date FROM Users;"
df_users = pd.read_sql(query_users, engine)
df_users["gender"] = df_users["gender"].map({"M": 0, "F": 1, "U": 2})

# 年齢の計算
df_users["age"] = pd.to_datetime("today").year - pd.to_datetime(df_users["birth_date"]).dt.year
df_users.drop(columns=["birth_date"], inplace=True)

# 年齢の画像表示 Min-Maxスケーリング
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df_users["age"] = scaler.fit_transform(df_users[["age"]])

df_users = pd.get_dummies(df_users, columns=["postal_code"])
area_columns = [col for col in df_users.columns if col.startswith("postal_code")]
df_users[area_columns] = df_users[area_columns].astype(int)


#UserTags情報の取得
query_tags = """SELECT u.user_id, t.tag_id FROM UserTags u JOIN Tags t ON u.tag_id = t.tag_id;"""
df_tags = pd.read_sql(query_tags, engine)
# ワンホットエンコーディング
df_tags_onehot = df_tags.pivot_table(index="user_id", columns="tag_id", aggfunc="size", fill_value=0)
df_tags_onehot = df_tags_onehot.add_prefix("tag_")

#PointTransaction情報の取得
query_transactions = "SELECT user_id, store_id FROM pointtransaction WHERE user_id IS NOT NULL;"
df_pointtransaction = pd.read_sql(query_transactions, engine)
# ワンホットエンコーディング
df_transactions_onehot = df_pointtransaction.pivot_table(index="user_id", columns="store_id", aggfunc="size", fill_value=0)
df_transactions_onehot = df_transactions_onehot.add_prefix("store_")


#データの統合
df_final = df_users.set_index("user_id")\
    .join(df_tags_onehot, how="left")\
    .join(df_transactions_onehot, how="left")\
    .fillna(0)

print(df_final.head())

# コサイン類似度の計算
cosine_sim = cosine_similarity(df_final)
# 結果をデータフレームに変換
cosine_sim_df = pd.DataFrame(cosine_sim, index=df_final.index, columns=df_final.index)
# コサイン類似度行列の表示
print(cosine_sim_df)

#類似ユーザーの抽出test
target_user_id = 1

# 類似ユーザーの抽出
def get_similar_users(user_id, top_n=5):
    return cosine_sim_df[user_id].sort_values(ascending=False).iloc[1:top_n+1]

# 類似ユーザーが訪問した店舗・イベントを集計
similar_users = get_similar_users(target_user_id).index
existing_users = [user for user in similar_users if user in df_transactions_onehot.index] # df_transactions_onehot に存在するユーザーだけフィルタ
if existing_users:
    recommended_stores = df_transactions_onehot.loc[existing_users].sum().sort_values(ascending=False)
    print(f"ユーザー {target_user_id} におすすめの店舗:")
    print(recommended_stores.head())
else:
    print(f"ユーザー {target_user_id} に類似ユーザーの来店履歴がありません。")


         gender  relationship_id       age  tag_1  tag_2  tag_3  tag_4  tag_5  \
user_id                                                                         
1             0                1  0.890625      1      0      0      0      0   
2             0                1  0.953125      0      0      1      0      0   
3             0                1  1.000000      0      0      0      0      1   
4             1                2  0.859375      0      0      0      0      0   
5             1                2  0.984375      0      0      0      0      0   

         tag_6  tag_7  ...  tag_23  tag_24  tag_25  tag_26  store_1  store_2  \
user_id                ...                                                     
1            0      0  ...       0       1       0       0      5.0      0.0   
2            0      0  ...       0       0       0       0      0.0      3.0   
3            0      0  ...       0       0       0       0      3.0      0.0   
4            1      1  ...      